In [2]:
pip install lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [197]:
import pandas as pd
from scipy import signal
import numpy as np
import matplotlib as plt
import time

import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings("ignore")

In [198]:
Combine = pd.read_csv("data/combine.csv")
Team_Stats = pd.read_csv("data/team_stats_2003_2023.csv")

In [199]:
Team_Stats = Team_Stats[Team_Stats["year"] != 2023]

## Webscrape all of the years for from pro-football reference that we are using

In [200]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = Team_Stats["year"].unique() + 1
years = years
positions = ['passing', 'rushing', 'receiving']
years

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [221]:
for pos in positions:
    print(pos)
    for year in years:
        link = website_begining + str(year) +'/'+ pos + '.htm'
        
        df = pd.read_html(link)
        
        csv_name = 'data/'+str(year) + '_' + pos + '.csv'
        df[0].to_csv(csv_name, sep=',', index=False, encoding='utf-8')
        time.sleep(4.2) #wait four seconds
        print(year)

passing
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
rushing
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
receiving
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


## Concat the tables together to put them all into one data frame and take out unnessary data from player name

In [201]:
passing = pd.read_csv('data/2004_passing.csv')

passing["Year"] = 2004
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_passing.csv')
    temp["Year"] = year
    passing = pd.concat([passing, temp])
    
passing = passing[passing["Pos"] == "QB"]


In [202]:
index = 0
for name in passing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    passing["Player"][index] = front + back
    
    index += 1

In [203]:
rushing = pd.read_csv('data/2004_rushing.csv')

rushing["Year"] = 2004
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_rushing.csv')
    temp["Year"] = year
    rushing = pd.concat([rushing, temp])
    
    
rushing.columns = rushing.iloc[0]
rushing = rushing.drop(0)
rushing.rename(columns={2004: 'Year'}, inplace=True)

rushing = rushing[rushing["Pos"] == "RB"]

In [204]:
index = 0
for name in rushing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    rushing["Player"][index] = front + back
    
    index += 1


In [205]:
receiving = pd.read_csv('data/2004_receiving.csv')

receiving["Year"] = 2004
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_receiving.csv')
    temp["Year"] = year
    receiving = pd.concat([receiving, temp])
    
receiving = receiving[receiving["Pos"] == "WR"]

In [206]:
index = 0
for name in receiving["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    receiving["Player"][index] = front + back
    
    index += 1
    

## Putting Combine Data into Positional Dfs

In [207]:
for col in Combine.columns:
    passing[col] = 0
    rushing[col] = 0
    receiving[col] = 0

In [208]:
passing_names = passing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in passing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        passing.loc[passing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(passing_names))) + "% of the players had Combine data")
passing = passing[passing["hsCountry"] != 0]

77% of the players had Combine data


In [209]:
rushing_names = rushing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in rushing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        rushing.loc[rushing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(rushing_names))) + "% of the players had Combine data")
rushing = rushing[rushing["hsCountry"] != 0]

68% of the players had Combine data


In [210]:
receiving_names = receiving["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in receiving_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        receiving.loc[receiving["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(receiving_names))) + "% of the players had Combine data")
receiving = receiving[receiving["hsCountry"] != 0]

58% of the players had Combine data


In [211]:
passing.drop(columns=['Rk'], inplace=True)
rushing.drop(columns=['Rk'], inplace=True)
receiving.drop(columns=['Rk'], inplace=True)

## Putting the team data into Positional Dfs

In [212]:
index = 0
for team in Team_Stats["team"]:
    team_split = team.split(" ")
    simple = team_split[-1]
    
    Team_Stats["team"][index] = simple
    index += 1

In [213]:
Team_Stats["team"] = Team_Stats["team"].replace("Redskins", "Commanders")
Team_Stats["team"] = Team_Stats["team"].replace("Team", "Commanders")

In [214]:
df_list = [passing, rushing, receiving]

In [215]:
for df in df_list:
    df["Tm"] = df["Tm"].replace("MIN", "Vikings")
    df["Tm"] = df["Tm"].replace("KAN", "Chiefs")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("DEN", "Broncos")
    df["Tm"] = df["Tm"].replace("GNB", "Packers")
    df["Tm"] = df["Tm"].replace("STL", "Rams")#
    df["Tm"] = df["Tm"].replace("PHI", "Eagles")
    df["Tm"] = df["Tm"].replace("NOR", "Steelers")#
    df["Tm"] = df["Tm"].replace("NWE", "Patriots")
    df["Tm"] = df["Tm"].replace("DAL", "Cowboys")

    df["Tm"] = df["Tm"].replace("HOU", "Texans")
    df["Tm"] = df["Tm"].replace("OAK", "Raiders")
    df["Tm"] = df["Tm"].replace("SDG", "Chargers")
    df["Tm"] = df["Tm"].replace("DET", "Lions")
    df["Tm"] = df["Tm"].replace("JAX", "Jaguars")
    df["Tm"] = df["Tm"].replace("BUF", "Bills")
    df["Tm"] = df["Tm"].replace("CIN", "Bengals")
    df["Tm"] = df["Tm"].replace("NYJ", "Jets")
    df["Tm"] = df["Tm"].replace("PIT", "Steelers")
    df["Tm"] = df["Tm"].replace("BAL", "Ravens")

    df["Tm"] = df["Tm"].replace("ARI", "Cardinals")
    df["Tm"] = df["Tm"].replace("ATL", "Falcons")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("SFO", "49ers")
    df["Tm"] = df["Tm"].replace("CLE", "Browns")
    df["Tm"] = df["Tm"].replace("TEN", "Titans")
    df["Tm"] = df["Tm"].replace("WAS", "Commanders")
    df["Tm"] = df["Tm"].replace("NYG", "Giants")
    df["Tm"] = df["Tm"].replace("CHI", "Bears")
    df["Tm"] = df["Tm"].replace("TAM", "Buccaneers")

    df["Tm"] = df["Tm"].replace("MIA", "Dolphins")
    df["Tm"] = df["Tm"].replace("CAR", "Panthers")
    df["Tm"] = df["Tm"].replace("SEA", "Seahawks")
    df["Tm"] = df["Tm"].replace("2TM", "Take_out")
    df["Tm"] = df["Tm"].replace("LAR", "Rams")
    df["Tm"] = df["Tm"].replace("LAC", "Chargers")
    df["Tm"] = df["Tm"].replace("LVR", "Raiders")
    
    df["Tm"] = df["Tm"].replace("Team", "Commanders")

In [216]:
passing = passing[(passing["Tm"] != "Take_out") & (passing["Tm"] != "3TM")]
rushing = rushing[(rushing["Tm"] != "Take_out") & (rushing["Tm"] != "3TM" )]
receiving = receiving[(receiving["Tm"] != "Take_out") & (receiving["Tm"] != "3TM")]

In [217]:
pos_team_cols = Team_Stats.columns[2:]
for col in pos_team_cols:
    passing["team_" + col] = 0
    rushing["team_" + col] = 0
    receiving["team_" + col] = 0
    

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

In [218]:
df_list = [passing, rushing, receiving]
team_years = Team_Stats["year"].unique()
for df in df_list:
    for year in team_years:
        for team in Team_Stats["team"].unique():
            team_values = Team_Stats[(Team_Stats["team"]==team) & (Team_Stats["year"]==year)].values.tolist()[0]
            df.loc[(df["Tm"] == team) & (df["Year"] == year + 1), "team_" + Team_Stats.columns[2:]] = team_values[2:]

## Taking out unnessary Columns

In [258]:
df_list = [passing, rushing, receiving]
questionable_cols = []
for df in df_list:
    question = 0
    question_temp = []
    for col in df.columns:
        if type(df[col][0]) != float or type(df[col][0]) != int:
            if col != 'Player' and col != 'Tm':
                try:
                    df[col] = df[col].astype(float)
                except:
                    question_temp.append(col)
                    question += 1

    print(100*question/len(df.columns), "% was questionable")
    print(question_temp)
    questionable_cols.append(question_temp)

0.0 % was questionable
[]
0.0 % was questionable
[]
0.0 % was questionable
[]


In [257]:
passing = passing.drop(columns = [questionable_cols[0]][0])
rushing = rushing.drop(columns = [questionable_cols[1]][0])
receiving = receiving.drop(columns = [questionable_cols[2]][0])

## Deal with Nan values

In [277]:
df_list = [passing, rushing, receiving]
nan_columns = []
for df in df_list:
    temp_nan = []
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage >= 0.05:
            print(col, column_nan_percentage*100)
            total += 1
            temp_nan.append(col)
    nan_columns.append(temp_nan)
    print(total/len(df.columns))

0.0
0.0
0.0


In [276]:
passing = passing.drop(columns = [nan_columns[0]][0])
rushing = rushing.drop(columns = [nan_columns[1]][0])
receiving = receiving.drop(columns = [nan_columns[2]][0])

## Replace rest of Nan with column average

In [281]:
df_list = [passing, rushing, receiving]
for df in df_list:
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage > 0:
            
            column_average = df[col].mean()


            df[col] = df[col].fillna(column_average)
            print(col)

In [291]:
print("Final Shapes of DataFrames:\n")
print('Passing Observations:', passing.shape[0], '       Passing Parameters:', passing.shape[1] - 2, '\n')
print('Rushing Observations:', rushing.shape[0], '       Rushing Parameters:', rushing.shape[1] - 2, '\n')
print('Receiving Observations:', receiving.shape[0], '     Receiving Parameters:', receiving.shape[1] - 2, '\n')

Final Shapes of DataFrames:

Passing Observations: 1120        Passing Parameters: 66 

Rushing Observations: 1875        Rushing Parameters: 53 

Receiving Observations: 2501      Receiving Parameters: 57 



In [292]:
passing.to_csv('data/Cleaned_Passing.csv', sep=',', index=False, encoding='utf-8')
rushing.to_csv('data/Cleaned_Rushing.csv', sep=',', index=False, encoding='utf-8')
receiving.to_csv('data/Cleaned_Receiving.csv', sep=',', index=False, encoding='utf-8')